# 1. CARGO DATAFRAMES BASE

In [1]:
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta

ruta_export = r"C:\Users\MSII7\Documents\Jasef\Desercion\export"
ruta_output = r"C:\Users\MSII7\Documents\Jasef\Desercion\output"
ruta_sources = r"C:\Users\MSII7\Documents\Jasef\Desercion\sources"

df_matriculados_2023_2 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2023_2.csv'))
df_matriculados_2023_1 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2023_1.csv'))
df_matriculados_2022_2 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2022_2.csv'))
df_matriculados_2022_1 = pd.read_csv(os.path.join(ruta_export,'Matriculados_2022_1.csv'))
df_desertores_2023_2 = pd.read_csv(os.path.join(ruta_export,'Desertores_2023_2.csv'))
df_variables = pd.read_csv(os.path.join(ruta_export,'DataMaestra_Estudiante.csv'))
df_notas_2023_2 = pd.read_csv(os.path.join(ruta_sources,'DATA_DETALLE_NOTAS-2023-2.csv'))

In [2]:
df_cant_desaprobado_agrupado = df_notas_2023_2.groupby('IdAlumno').agg(
    CantParcialesDesaprobados=(
        'Actividad', 
        lambda x: ((x == 'EVALUACION PARCIAL') & (df_notas_2023_2.loc[x.index, 'NotaActiv'] < 11)).sum()
    ),
    CantFinalesDesaprobados=(
        'Actividad', 
        lambda x: ((x == 'EVALUACION FINAL') & (df_notas_2023_2.loc[x.index, 'NotaActiv'] < 11)).sum()
    )
).reset_index()

print(df_cant_desaprobado_agrupado.describe)

<bound method NDFrame.describe of          IdAlumno  CantParcialesDesaprobados  CantFinalesDesaprobados
0       100002894                          0                        0
1       100003558                          2                        3
2       100003932                          0                        0
3       100003946                          1                        1
4       100003989                          0                        0
...           ...                        ...                      ...
14861  4200810288                          0                        0
14862  4201010209                          0                        0
14863  4201010528                          2                        3
14864  4201010550                          4                        4
14865  4201011098                          0                        0

[14866 rows x 3 columns]>


In [3]:
# Unir todos los periodos anteriores en un solo dataframe
df_matriculados_anteriores = pd.concat([df_matriculados_2023_1, df_matriculados_2022_2, df_matriculados_2022_1])

In [4]:
df_nuevos_matriculados_2023_2 = df_matriculados_2023_2[~df_matriculados_2023_2['IdAlumno'].isin(df_matriculados_anteriores['IdAlumno'])]

In [6]:
# Filtrar el DataFrame para encontrar el registro específico
resultado = df_nuevos_matriculados_2023_2[df_nuevos_matriculados_2023_2['IdAlumno'] == 100118394]

# Verifica el resultado
print(resultado)

Empty DataFrame
Columns: [CicloLectivoDesCorta, IdAlumno, Anio]
Index: []


In [7]:
print(df_nuevos_matriculados_2023_2.shape)

(1205, 3)


In [8]:
print(df_variables.isnull().sum())

IdAlumno                   0
Genero                     0
FechaNacimiento            0
Distrito                   0
Provincia                  0
Departamento               0
FamiliarResponsable    20211
AnioNacimiento             0
dtype: int64


In [9]:
#Variables CRM
df = pd.merge(df_nuevos_matriculados_2023_2, df_variables, on='IdAlumno', how='left')

#Parciales Desaprobados
df = pd.merge(df, df_cant_desaprobado_agrupado, on='IdAlumno', how='left')

# columna objetivo
df['Desercion'] = df['IdAlumno'].isin(df_desertores_2023_2['IdAlumno']).astype(int)

print(df.head())

  CicloLectivoDesCorta   IdAlumno  Anio Genero FechaNacimiento Distrito  \
0               2023-2  100004070  2023    NaN             NaN      NaN   
1               2023-2  100008630  2023    NaN             NaN      NaN   
2               2023-2  100009785  2023    NaN             NaN      NaN   
3               2023-2  100011191  2023    NaN             NaN      NaN   
4               2023-2  100012563  2023    NaN             NaN      NaN   

  Provincia Departamento FamiliarResponsable  AnioNacimiento  \
0       NaN          NaN                 NaN             NaN   
1       NaN          NaN                 NaN             NaN   
2       NaN          NaN                 NaN             NaN   
3       NaN          NaN                 NaN             NaN   
4       NaN          NaN                 NaN             NaN   

   CantParcialesDesaprobados  CantFinalesDesaprobados  Desercion  
0                          0                        0          0  
1                          0  

### RELLENO LOS QUE NO TIENEN GENERO



In [10]:
# Calculamos la cuenta de cada género y de los valores nulos
conteo_generos = df['Genero'].value_counts(dropna=False)
count_f = conteo_generos.get('F', 0)  # Cantidad de F
count_m = conteo_generos.get('M', 0)  # Cantidad de M
total_blancos = conteo_generos.get(np.nan, 0)  # Cantidad de valores nulos

# Ahora, calculamos la distribución equitativa
if total_blancos > 0:
    # Proporciones de cada género
    proporciones_f = count_f / (count_f + count_m)
    proporciones_m = count_m / (count_f + count_m)

    # Cálculo de cuántos géneros se asignarán a los valores nulos
    asignacion_f = int(total_blancos * proporciones_f)
    asignacion_m = total_blancos - asignacion_f  # Lo que queda se asigna a M

    # Rellenar los valores nulos en el DataFrame
    df.loc[df['Genero'].isna(), 'Genero'] = ['F'] * asignacion_f + ['M'] * asignacion_m


print(df.isnull().sum())

CicloLectivoDesCorta           0
IdAlumno                       0
Anio                           0
Genero                         0
FechaNacimiento               90
Distrito                      90
Provincia                     90
Departamento                  90
FamiliarResponsable          742
AnioNacimiento                90
CantParcialesDesaprobados      0
CantFinalesDesaprobados        0
Desercion                      0
dtype: int64


### RELLENO LOS QUE NO TIENEN FECHA DE NACIMIENTO

In [11]:
# Contar cuántos valores nulos hay en FechaNacimiento
n = df['FechaNacimiento'].isnull().sum()

# Función para generar fechas aleatorias en el formato 'YYYY-MM-DD'
def generar_fecha_aleatoria(n):
    fechas = []
    for _ in range(n):
        anio = np.random.choice([2006, 2007])  # Años para tener 16 o 17 años en 2023
        mes = np.random.randint(1, 13)  # Mes de 1 a 12
        dia = np.random.randint(1, 29)  # Día de 1 a 28 (para simplificar)
        fecha = f"{anio}-{mes:02d}-{dia:02d}"  # Formato 'YYYY-MM-DD'
        fechas.append(fecha)
    return fechas

# Generar fechas aleatorias y rellenar los nulos
fechas_aleatorias = generar_fecha_aleatoria(n)
df.loc[df['FechaNacimiento'].isnull(), 'FechaNacimiento'] = fechas_aleatorias

# Obtener el año de nacimiento de los nulos en FechaNacimiento
df.loc[df['AnioNacimiento'].isnull(), 'AnioNacimiento'] = df['FechaNacimiento'].str.split('-').str[0].astype(int)

#print(df)
print(df.isnull().sum())


CicloLectivoDesCorta           0
IdAlumno                       0
Anio                           0
Genero                         0
FechaNacimiento                0
Distrito                      90
Provincia                     90
Departamento                  90
FamiliarResponsable          742
AnioNacimiento                 0
CantParcialesDesaprobados      0
CantFinalesDesaprobados        0
Desercion                      0
dtype: int64


### RELLENO LOS QUE NO TIENEN DISTRITO

In [12]:
# Lista de distritos para rellenar
distritos = ["ATE", "VILLA EL SALVADOR", "COMAS", "CHORRILLOS", "SAN MARTÍN DE PORRES"]

# Contar cuántos valores nulos hay en el campo 'Distrito'
nulos_distrito = df['Distrito'].isnull().sum()

# Calcular cuántos registros de cada distrito se necesitan
if nulos_distrito > 0:
    # Calcular cuántos distritos asignar a cada uno
    asignaciones = [nulos_distrito // len(distritos)] * len(distritos)
    
    # Distribuir los restantes de manera aleatoria entre los distritos
    for i in range(nulos_distrito % len(distritos)):
        asignaciones[i] += 1

    # Crear una lista con los distritos asignados
    distritos_asignados = []
    for distrito, cantidad in zip(distritos, asignaciones):
        distritos_asignados.extend([distrito] * cantidad)

    # Mezclar aleatoriamente la lista para distribuir uniformemente
    np.random.shuffle(distritos_asignados)

    # Rellenar los valores nulos en el DataFrame
    df.loc[df['Distrito'].isnull(), 'Distrito'] = distritos_asignados

# Mostrar el DataFrame resultante
print(df.isnull().sum())

CicloLectivoDesCorta           0
IdAlumno                       0
Anio                           0
Genero                         0
FechaNacimiento                0
Distrito                       0
Provincia                     90
Departamento                  90
FamiliarResponsable          742
AnioNacimiento                 0
CantParcialesDesaprobados      0
CantFinalesDesaprobados        0
Desercion                      0
dtype: int64


### RELLENO LOS QUE NO TIENEN PROVINCIA NI DEPARTAMENTO

In [13]:
df['Provincia'] = df['Provincia'].fillna('LIMA')
df['Departamento'] = df['Departamento'].fillna('LIMA')

print(df.isnull().sum())

CicloLectivoDesCorta           0
IdAlumno                       0
Anio                           0
Genero                         0
FechaNacimiento                0
Distrito                       0
Provincia                      0
Departamento                   0
FamiliarResponsable          742
AnioNacimiento                 0
CantParcialesDesaprobados      0
CantFinalesDesaprobados        0
Desercion                      0
dtype: int64


### CREO CAMPO EDAD

In [14]:
df['Edad'] = df['Anio'] - df['AnioNacimiento']
print(df.isnull().sum())

CicloLectivoDesCorta           0
IdAlumno                       0
Anio                           0
Genero                         0
FechaNacimiento                0
Distrito                       0
Provincia                      0
Departamento                   0
FamiliarResponsable          742
AnioNacimiento                 0
CantParcialesDesaprobados      0
CantFinalesDesaprobados        0
Desercion                      0
Edad                           0
dtype: int64


### RELLENO EL FAMILIAR RESPONSABLE

In [15]:
# Filas donde FamiliarResponsable es nulo
mask = df['FamiliarResponsable'].isnull()

# Aplicar la condición solo a esas filas
df.loc[mask, 'FamiliarResponsable'] = df.loc[mask, 'Edad'].apply(lambda x: 'Apoderado' if x <= 18 else 'Alumno')

print(df.isnull().sum())

CicloLectivoDesCorta         0
IdAlumno                     0
Anio                         0
Genero                       0
FechaNacimiento              0
Distrito                     0
Provincia                    0
Departamento                 0
FamiliarResponsable          0
AnioNacimiento               0
CantParcialesDesaprobados    0
CantFinalesDesaprobados      0
Desercion                    0
Edad                         0
dtype: int64


In [16]:
variables_modelo = ['Edad', 'Genero', 'Distrito', 'Provincia', 'Departamento', 'FamiliarResponsable', 'CantParcialesDesaprobados', 'CantFinalesDesaprobados', 'Desercion']
df_modelo = df[variables_modelo]

In [17]:
df_modelo = pd.get_dummies(df_modelo, columns=['Genero', 'Distrito', 'Provincia', 'Departamento', 'FamiliarResponsable', 'CantParcialesDesaprobados', 'CantFinalesDesaprobados'], drop_first=True)

# CREACION DEL MODELO

In [18]:
#División de los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

X = df_modelo.drop(columns=['Desercion'])
y = df_modelo['Desercion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
# Diccionario que guarda las métricas
metricas_dict = {
    "Modelo": [],
    "Tipo_Data": [],
    "Exactitud": [],
    "AUC": [],
    "F1-Score": [],
    "Precisión": [],
    "Recall": [],
    #"Verdaderos Negativos": [],
    #"Falsos Positivos": [],
    #"Falsos Negativos": [],
    #Verdaderos Positivos": [],
    "Puntuación": []
}

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix,f1_score, precision_score, recall_score
import xgboost as xgb

# Función para calcular métricas
def calcular_metricas(model, X_test, y_test, nombre_modelo):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    puntuacion = (0.1 * accuracy) + (0.2 * precision) + (0.5 * recall) + (0.3 * f1) + (0.2 * auc)
    
    metricas_dict["Modelo"].append(nombre_modelo)
    metricas_dict["Tipo_Data"].append("Data Original")
    metricas_dict["Exactitud"].append(accuracy)
    metricas_dict["AUC"].append(auc)
    metricas_dict["F1-Score"].append(f1)
    metricas_dict["Precisión"].append(precision)
    metricas_dict["Recall"].append(recall)
    metricas_dict["Puntuación"].append((puntuacion))

## MODELOS SIN BALANCEO

In [21]:
modelos = [
    ("Regresión Logística", LogisticRegression(max_iter=1000)),
    ("Random Forest", RandomForestClassifier(random_state=42)),
    ("XGBoost", xgb.XGBClassifier(eval_metric='logloss', random_state=42)),
    ("SVM", SVC(probability=True, random_state=42)),
    ("KNN",KNeighborsClassifier(n_neighbors=5))
]

In [22]:
# Entrenar y evaluar los modelos
for nombre_modelo, modelo in modelos:
    modelo.fit(X_train, y_train)
    calcular_metricas(modelo, X_test, y_test, nombre_modelo)

## MODELOS CON BALANCEO

In [23]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Reentrenar modelos con datos balanceados
for nombre_modelo, modelo in modelos:
    modelo.fit(X_resampled, y_resampled)
    calcular_metricas(modelo, X_test, y_test, f"{nombre_modelo} (SMOTE)")

## MODELOS CON BALANCEO Y AJUSTE DE HIPERPARAMETROS

In [24]:
from sklearn.model_selection import GridSearchCV

# Definir los hiperparámetros para cada modelo
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['lbfgs', 'liblinear']
}

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

param_grid_svm = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance']
}

param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7]
}

In [25]:
# Diccionario con modelos y sus respectivos grids
modelos_param_grids = [
    ("Regresión Logística", LogisticRegression(max_iter=1000), param_grid_lr),
    ("Random Forest", RandomForestClassifier(random_state=42), param_grid_rf),
    ("XGBoost", xgb.XGBClassifier(eval_metric='logloss', random_state=42), param_grid_xgb),
    ("SVM", SVC(probability=True, random_state=42), param_grid_svm),
    ("KNN", KNeighborsClassifier(), param_grid_knn)
]

In [26]:
# Realizamos la búsqueda de hiperparámetros
for nombre_modelo, modelo, param_grid in modelos_param_grids:
    grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid, scoring='roc_auc', cv=5, n_jobs=-1)
    grid_search.fit(X_resampled, y_resampled)
    
    # Entrenamos con los mejores hiperparámetros
    best_model = grid_search.best_estimator_
    print(f"Mejores parámetros para {nombre_modelo}: {grid_search.best_params_}")
    
    # Calculamos métricas con los mejores hiperparámetros
    calcular_metricas(best_model, X_test, y_test, f"{nombre_modelo} (Optimizado y con SMOTE)")

Mejores parámetros para Regresión Logística: {'C': 10, 'solver': 'liblinear'}
Mejores parámetros para Random Forest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 300}
Mejores parámetros para XGBoost: {'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 100}
Mejores parámetros para SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Mejores parámetros para KNN: {'n_neighbors': 5, 'weights': 'distance'}


# EXPORTADO DE METRICAS

In [27]:
df_metricas = pd.DataFrame(metricas_dict)

df_metricas = df_metricas.sort_values(by="Puntuación", ascending=False)
df_metricas.to_csv(os.path.join(ruta_output,'Modelo_Desercion_Metricas.csv'))
print(df_metricas)

                                          Modelo      Tipo_Data  Exactitud  \
10  Regresión Logística (Optimizado y con SMOTE)  Data Original   0.781768   
13                  SVM (Optimizado y con SMOTE)  Data Original   0.767956   
5                    Regresión Logística (SMOTE)  Data Original   0.745856   
0                            Regresión Logística  Data Original   0.754144   
12              XGBoost (Optimizado y con SMOTE)  Data Original   0.715470   
11        Random Forest (Optimizado y con SMOTE)  Data Original   0.729282   
7                                XGBoost (SMOTE)  Data Original   0.707182   
2                                        XGBoost  Data Original   0.715470   
6                          Random Forest (SMOTE)  Data Original   0.701657   
1                                  Random Forest  Data Original   0.704420   
14                  KNN (Optimizado y con SMOTE)  Data Original   0.676796   
9                                    KNN (SMOTE)  Data Original 